## Model - XLNet

XLNet is a transformer model that is very effective at learning the context in a text. Instead of the conventional sequence-to-sequence training, it uses a permutation-based approach, which allows it to collect context from across a sentence. This is especially useful for examining longer, more complicated reviews. For similar reason that it is already pretrained and adapts easily to task like sentiment analysis without much training from us.

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW,get_scheduler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
# Loading the train dataset
df_train = pd.read_csv("processed_test.csv")
df_test = pd.read_csv("processed_test.csv")

In [ ]:
df_train.isnull().sum()

sentiment           0
processed_review    0
dtype: int64

In [ ]:
blank_counts = (df_train.isna() | df_train.applymap(lambda x: isinstance(x, str) and x.strip() == '')).sum()
print(blank_counts)

sentiment           0
processed_review    0
dtype: int64


/tmp/ipykernel_433/313185806.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  blank_counts = (df_train.isna() | df_train.applymap(lambda x: isinstance(x, str) and x.strip() == '')).sum()


In [ ]:
df_test.isnull().sum()

sentiment           0
processed_review    0
dtype: int64

In [ ]:
blank_counts = (df_test.isna() | df_test.applymap(lambda x: isinstance(x, str) and x.strip() == '')).sum()
print(blank_counts)


sentiment           0
processed_review    0
dtype: int64


/tmp/ipykernel_433/2172214410.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  blank_counts = (df_test.isna() | df_test.applymap(lambda x: isinstance(x, str) and x.strip() == '')).sum()


In [ ]:
# Ensuring the correct data types
df_train['sentiment'] = df_train['sentiment'].astype(int)
df_test['sentiment'] = df_test['sentiment'].astype(int)

In [ ]:
# Assuming the dataset has 'processed_review' and 'sentiment' columns
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])  # Ensuring the text is a string
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install sentencepiece==0.1.96

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip show sentencepiece

Name: sentencepiece
Version: 0.1.96
Summary: SentencePiece python wrapper
Home-page: https://github.com/google/sentencepiece
Author: Taku Kudo
Author-email: taku@google.com
License: Apache
Location: /opt/conda/lib/python3.10/site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import XLNetTokenizer

# Initializing the tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

In [ ]:
# Preparing the dataset
dataset_train = TextDataset(df_train['processed_review'].tolist(), df_train['sentiment'].tolist(), tokenizer)
dataset_test = TextDataset(df_test['processed_review'].tolist(), df_test['sentiment'].tolist(), tokenizer)

In [ ]:
# DataLoader
train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False)

In [ ]:
# Loading the XLNet model
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0-11): 12 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (sequence_summary): SequenceSummary(
    (summary): Linear(in_features=768, out_features=768, bias=True)
    (activation): Tanh()
    (first_dropout): Identity()
    (last

In [ ]:
# Defining the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 3)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def train_model(model, train_loader, optimizer, criterion, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}")
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()

            progress_bar.set_postfix(loss=total_loss / len(train_loader))

        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader):.4f}")

    torch.save(model.state_dict(), "trained_xlnet_model.pt")
    print("Model saved successfully!")

# Training the model
train_model(model, train_loader, optimizer, criterion, scheduler)

Epoch 1: 100%|██████████| 2500/2500 [02:55<00:00, 14.27it/s, loss=0.242]


Epoch 1, Loss: 0.2422


Epoch 2: 100%|██████████| 2500/2500 [02:53<00:00, 14.40it/s, loss=0.145]


Epoch 2, Loss: 0.1447


Epoch 3: 100%|██████████| 2500/2500 [02:53<00:00, 14.41it/s, loss=0.0822]


Epoch 3, Loss: 0.0822
Model saved successfully!


In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        progress_bar = tqdm(test_loader, desc="Evaluating")
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

    print(f'\n Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')

In [ ]:
# Evaluating the model
evaluate_model(model, test_loader)

Evaluating: 100%|██████████| 2500/2500 [01:05<00:00, 38.03it/s]


 Accuracy: 0.9896
Precision: 0.9896
Recall: 0.9896
F1 Score: 0.9896
